# Data Analysis Case Study - Apply Problem Solving Strategy

**Problem**
Assume you are in the role of an Analyst in the product development of the company. To help increase the volume of orders and customers purchasing movie tickets online, you need to analyze data of customers' booking history over the past years. From there, provide informative insights about customer behavior and corresponding recommendations.

This is a capstone project I completed for self-learning Python for Data Analysis, utilizing industry-standard libraries: Pandas and Matplotlib.
There are many frameworks for data analysis. Framework provides a structured and systemetic approach for managing and analyzing data tasks/projects to ensure consistency, efficiency, and high quality work. The framework I used is: 

Problem Definition => Data Collection => Data Transformation => Data Exploration => Sharing Insights

## I. Define The Problem 
### Why?
- The objective of this task is to analyze user behavior in purchasing movie tickets over the past four years using data.
- By gaining this understanding, we can develop insights to implement effective strategies to increase sales and the number of buyers in the coming years.

### Who?
- Internal: Departments and/or staffs involves in ticket purchasing process? Sales, customer service
- External: Who is our customers?
-  Location: which city/province/area
-  Profile: new customer, old customer, activated customer,..
-  Demographic: gender (male/femail), education, marriage or not,..

### What?
- Analyze ticket purchasing behavior using website and mobile app

### Which
Which element or factor  does customers use duing online ticket purchasing process?
-- Product: tickets
-- Device: phone, website
-- Payment options: cash, card, bank transfer, e-wallet 
-- Price
-- Promotion / Discount

### When
When do customers often buy tickets?
- Year, month, day, hour
- Holiday, event, new movie

### How
How did customers use the product?
- Customer experience: Good, bad
++ Feedback
++ %Successful ticket purchases
++ %Customer retention
- Purchasing process
++ Total time completing ticket purchase
++ Time taken for each step in the process. Any bottle-necks? Any step that customer faced frustration?

# II. Disaggregate The Problem
Based on the problem definition, we can create a Logic Tree

## 1. Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_customer = pd.read_csv("D:/Git DucD Data/SQLplaygroundDataAnalysis/Capstone project/Dataset/customer.csv")
df_campaign = pd.read_csv("D:/Git DucD Data/SQLplaygroundDataAnalysis/Capstone project/Dataset/campaign.csv")
df_device = pd.read_csv("D:/Git DucD Data/SQLplaygroundDataAnalysis/Capstone project/Dataset/device_detail.csv")
df_sales = pd.read_csv("D:/Git DucD Data/SQLplaygroundDataAnalysis/Capstone project/Dataset/sales.csv")
df_status = pd.read_csv("D:/Git DucD Data/SQLplaygroundDataAnalysis/Capstone project/Dataset/status_detail.csv")
df_ticket = pd.read_csv("D:/Git DucD Data/SQLplaygroundDataAnalysis/Capstone project/Dataset/ticket_history.csv")

## 2. Clean Data

2.1 Data type, NULL value, duplicate values

In [13]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52054 entries, 0 to 52053
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     52054 non-null  int64 
 1   usergender  52054 non-null  int64 
 2   dob         52054 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.2+ MB
